In [ ]:
!pip install pycaret
!pip install geopandas
!pip install geemap

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
import pycaret as car
import geemap
import ee
import eeconvert

from functools import reduce
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
grid = ee.FeatureCollection('users/geethensingh/WTWTA_grid75457')
print(grid.size().getInfo())
# Map.addLayer(grid)

grid_img = grid.reduceToImage(**{'properties': ['fid'], 'reducer': ee.Reducer.first()}).rename('fid')
# Map.addLayer(grid_img)

6


In [ ]:
label = ee.ImageCollection.fromImages([ee.Image('users/jdmwhite/UD_buf_mean'),ee.Image('users/geethensingh/UD_buf_GPS')]).mean().rename('label')
covs = ee.Image('users/jdmwhite/WTWTA_covariates_1000_sel').addBands([grid_img, label])
# covs = ee.Image('users/jdmwhite/WTWTA_covariates_1000_sel').addBands([label])
# print(covs.bandNames().getInfo())
pts = ee.FeatureCollection('users/jdmwhite/buff_sampled_points').select([])
print(pts.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.758101185332148, -24.595835104973677]}, 'id': '00000000000000001d2b', 'properties': {}}


In [ ]:
print(pts.size().getInfo())
extracted_covs = covs.reduceRegions(**{'collection': pts, 'scale': 1000, 'reducer': ee.Reducer.first()})
print(extracted_covs.size().getInfo())

10491
10491


In [ ]:
def largefc_to_pandas(featurecollection, step_size = 5000):
    """
    function to convert a featurecollection with more than 5000 elements to a pandas dataframe
    
    featurecollection: earth engine feature collection with more than 5000 elements
    step_size: Th enumber of elemenst to process at once. Must be less than equal to 5000
    
    """
    df = pd.DataFrame()
    fclist = featurecollection.toList(featurecollection.size())
    fcsize = featurecollection.size().getInfo()
    for i in range(0,fcsize,step_size):
        fc = ee.FeatureCollection(fclist.slice(i,i+step_size))
        df = df.append(geemap.ee_to_pandas(fc))
    return df

In [ ]:
df  = largefc_to_pandas(extracted_covs)
print(f"There are {len(df)} features")
print(df.info(memory_usage='deep'))
print(df.head(5))

There are 10491 features
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10491 entries, 0 to 490
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b1      10116 non-null  float64
 1   b10     10116 non-null  float64
 2   b11     10116 non-null  float64
 3   b12     10116 non-null  float64
 4   b13     10116 non-null  float64
 5   b14     10116 non-null  float64
 6   b15     10116 non-null  float64
 7   b16     10116 non-null  float64
 8   b17     10116 non-null  float64
 9   b18     10116 non-null  float64
 10  b19     10116 non-null  float64
 11  b2      10116 non-null  float64
 12  b20     10116 non-null  float64
 13  b21     10116 non-null  float64
 14  b22     10027 non-null  float64
 15  b23     10116 non-null  float64
 16  b24     10116 non-null  float64
 17  b25     10116 non-null  float64
 18  b26     10116 non-null  float64
 19  b27     10116 non-null  float64
 20  b28     10116 non-null  float64
 21  b3      1011

In [ ]:
df_lgmb = df[['label','fid','b1','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b2','b20','b21', 'b22', 'b23', 'b24', 'b25', 'b26','b27','b28','b3','b32','b33','b34','b35','b36','b37','b38','b4','b43','b44','b45','b5','b6','b7','b8','b9','b29','b42']]

In [ ]:
df_lgmb.shape

(10491, 42)

In [ ]:
df_lgmb = df_lgmb.fillna(0)

Note: Need to specify the spatial fold in the setup?

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = df_lgmb, test_data = df_lgmb, target = 'label', fold = 6, fold_strategy = 'groupkfold', fold_groups = 'fid',\
categorical_imputation = 'constant', numeric_imputation = 'mean',\
session_id=42)

,Description,Value
0,session_id,42
1,Target,label
2,Original Data,"(10491, 42)"
3,Missing Values,True
4,Numeric Features,28
5,Categorical Features,13
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(7317, 168)"


In [ ]:
best = compare_models(exclude = errors="raise")

In [ ]:
compare_models()

[]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=232f2193-6834-457f-a7ff-77b7f2e92f86' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>